In [1]:
import pandas as pd
import json

prompts = pd.read_csv('fn1.7-test-prompts.csv')

# predictions = pd.read_json('batch-gpt-4o-mini-0.0temperature-json-existing-cot-predictions-cleaned.jsonl', lines=True)
with open('batch-gpt-4o-mini-0.0temperature-json-complete-cot-predictions-cleaned.jsonl') as f:
    predictions = f.readlines()

predictions = [json.loads(p.strip()) for p in predictions]

prompts['prediction'] = predictions
predictions = prompts[['output', 'prediction', 'frame_elements']]

In [2]:
tp = 0
near_miss = 0
fp = 0
fn = 0
tn = 0 # not really used because tags only have positive values, technically all other FEs are TN, but we don't really care about that (maybe we should?)

for label, pred, fes in predictions[['output', 'prediction', 'frame_elements']].values:

    # Get each predicted FE span from the prediction
    pred_tags = pred
    real_tags = eval(fes)

    # Check each predicted FE span
    for tag, content in pred_tags.items():
        if tag.capitalize() in real_tags:
            if content == real_tags[tag.capitalize()]:
                tp += 1
            else:
                fp += 1
                near_miss += 1
        else:
            fp += 1
    
    # Check each real FE span
    for tag, content in real_tags.items():
        if tag not in pred_tags:
            fn += 1
        elif content != pred_tags[tag]:
            fn += 1

# print(f'Perfect: {perfect} out of {len(predictions)}')
print(f'TP: {tp}')
print(f'Missed: {near_miss}')
print(f'FP: {fp}')
print(f'FN: {fn}')

print(f'Precision (ignoring near misses): {tp / (tp + fp - near_miss)}')
print(f'Precision: {tp / (tp + fp)}')
print(f'Recall: {tp / (tp + fn)}')
print(f'F1: {2 * tp / (2 * tp + fp + fn)}')

print(f'Accuracy: {tp / (tp + fn + fp)}')



TP: 6204
Missed: 4702
FP: 53852
FN: 4745
Precision (ignoring near misses): 0.11207862123785092
Precision: 0.10330358332223258
Recall: 0.5666270892318933
F1: 0.17474825716498837
Accuracy: 0.09573926328297402
